In [1]:
## Import Packages
from __future__ import print_function

import numpy as np
import pandas as pd
from itertools import product

#Astro Software
import astropy.units as units
from astropy.coordinates import SkyCoord
from astropy.io import fits

#Plotting Packages
import matplotlib as mpl
import matplotlib.cm as cmplt
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.colors import LogNorm
from matplotlib.colors import ListedColormap
from mpl_toolkits.axes_grid1 import make_axes_locatable

import seaborn as sns

from PIL import Image, ImageDraw

from yt.config import ytcfg
import yt
import yt.units as u

#Scattering NN
import torch
import torch.nn.functional as F
from torch import optim
from kymatio.torch import Scattering2D
device = "cpu"

#Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.decomposition import PCA, FastICA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

import skimage
from skimage import filters
from skimage.filters import window

from scipy.optimize import curve_fit
from scipy import linalg
from scipy import stats
from scipy.signal import general_gaussian
from scipy.ndimage import map_coordinates

# from scipy.sparse import csr_matrix
from scipy.sparse import csc_matrix
from scipy.sparse.csgraph import reverse_cuthill_mckee

#I/O
import h5py
import pickle
import glob
import copy
import time
import os
#from bs4 import BeautifulSoup
import scipy.io as sio

#Plotting Style
%matplotlib inline
plt.style.use('dark_background')
rcParams['text.usetex'] = False
rcParams['axes.titlesize'] = 20
rcParams['xtick.labelsize'] = 16
rcParams['ytick.labelsize'] = 16
rcParams['legend.fontsize'] = 12
rcParams['axes.labelsize'] = 20
rcParams['font.family'] = 'sans-serif'

#Threading
torch.set_num_threads=2
from multiprocessing import Pool

import ntpath
def path_leaf(path):
    head, tail = ntpath.split(path)
    out = os.path.splitext(tail)[0]
    return out

def hd5_open(file_name,name):
    f=h5py.File(file_name,'r', swmr=True)
    data = f[name][:]
    f.close()
    return data

from matplotlib.colors import LinearSegmentedColormap
cdict1 = {'red':   ((0.0, 0.0, 0.0),
                   (0.5, 0.0, 0.0),
                   (1.0, 1.0, 1.0)),

         'green': ((0.0, 0.0, 0.0),
                   (1.0, 0.0, 0.0)),

         'blue':  ((0.0, 0.0, 1.0),
                   (0.5, 0.0, 0.0),
                   (1.0, 0.0, 0.0))
        }
blue_red1 = LinearSegmentedColormap('BlueRed1', cdict1,N=5000)
blue_red1.set_bad(color='yellow')

In [2]:
def DHC_iso_all(wst,J,L):
    (nk, Nd) = np.shape(wst)
    S0 = wst[:,0:2]
    S1 = wst[:,2:J*L+2]
    S2 = np.transpose(np.reshape(wst[:,J*L+3:],(nk,(J*L+1),(J*L+1))),(0,2,1))
    
    S1iso = np.zeros((nk,J))
    for j1 in range(J):
        for l1 in range(L):
            S1iso[:,j1] += S1[:,l1*J+j1]
    
    S2iso = np.zeros((nk,J,J,L))
    for j1 in range(J):
        for j2 in range(J):
            for l1 in range(L):
                for l2 in range(L):
                    deltaL = np.mod(l1-l2,L)
                    S2iso[:,j1,j2,deltaL] += S2[:,l1*J+j1,l2*J+j2]
                    
    Sphi1 = np.zeros((nk,J))
    for j1 in range(J):
        for l1 in range(L):
            Sphi1[:,j1] += S2[:,l1*J+j1,L*J]
            
    Sphi2 = np.zeros((nk,J))
    for j1 in range(J):
        for l1 in range(L):
            Sphi2[:,j1] += S2[:,L*J,l1*J+j1]
            
    return np.hstack((S0,S1iso,wst[:,J*L+2].reshape(nk,1),S2iso.reshape(nk,int(J*J*L)),Sphi1,Sphi2,S2[:,L*J,L*J].reshape(nk,1)))

In [3]:
mnist_DHC_out = hd5_open('mnist_DHC_out.h5','main/data')
mnist_DHC_out_test = hd5_open('mnist_DHC_out_test.h5','main/data')

In [4]:
mnist_train_y = hd5_open('mnist_train_y.h5','main/data')
mnist_test_y = hd5_open('mnist_test_y.h5','main/data')

In [5]:
def prec_LDA(lda,X_test,y_test,X_train,y_train):
    y_pred = lda.predict(X_test)
    prec = precision_score(y_test,y_pred,average='micro')
    
    y_train_pred = lda.predict(X_train)
    prec_train = precision_score(y_train,y_train_pred,average='micro')
    return (prec, prec_train)

In [6]:
temp = DHC_iso_all(mnist_DHC_out,6,8)
temp_test = DHC_iso_all(mnist_DHC_out_test,6,8)

combo_X = np.vstack((temp,temp_test))
combo_Y = np.hstack((mnist_train_y,mnist_test_y))

In [7]:
def mask_prec_fn(j,mask,X_train,y_train,X_test,y_test):
    mask_loc = copy.deepcopy(mask)
    mask_loc[j] = False
    lda = LDA()
    lda.fit(X_train[:,mask_loc], y_train)
    prec, prec_train = prec_LDA(lda,X_test[:,mask_loc],y_test,X_train[:,mask_loc],y_train)
    return (j,prec, prec_train)

In [8]:
from functools import partial

In [9]:
import tqdm as tq

In [33]:
k=1
if __name__ == '__main__':
    pool = Pool(31)
    print(int(k*12153))
    X_train, X_test, y_train, y_test = train_test_split(
        combo_X, 
        combo_Y, 
        stratify=combo_Y, 
        train_size=0.8,
        test_size=0.2,
        random_state=int(k*12153))

    index_list = np.arange(310)

    prec_max = []
    drop_indx = []
    train_prec = []

    for i in tq.tqdm(range(309),leave=True):
        mask = np.ones(310, dtype=bool)
        mask[drop_indx] = False
        prec_temp = np.zeros(310)
        train_prec_temp = np.zeros(310)
        temp_fnc = partial(mask_prec_fn, mask=mask,X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test)
        out = pool.map(temp_fnc,index_list[mask])
        out1 = list(map(list, zip(*out)))
        
        for indx, ind in enumerate(out1[0]):
            prec_temp[ind] = out1[1][indx]
            train_prec_temp[ind] = out1[2][indx]
        
        arg = np.random.choice(np.flatnonzero(prec_temp == prec_temp.max()))
        drop_indx.append(arg)
        prec_arg = prec_temp[arg]
        prec_max.append(prec_arg)
        train_prec_arg = train_prec_temp[arg]
        train_prec.append(train_prec_arg)

        print(arg,prec_arg,train_prec_arg)
    k+=1
    pool.close()
    pool.join()

  0%|          | 0/309 [00:00<?, ?it/s]

12153


  0%|          | 1/309 [00:58<5:02:32, 58.94s/it]

274 0.9291428571428572 0.9290357142857143


  1%|          | 2/309 [01:57<5:01:12, 58.87s/it]

...

135 0.5527142857142857 0.556


100%|█████████▉| 308/309 [3:09:15<00:02,  2.72s/it]

124 0.43557142857142855 0.43373214285714284


100%|██████████| 309/309 [3:09:16<00:00, 36.75s/it]

236 0.29228571428571426 0.2924107142857143


In [34]:
hf = h5py.File('drop_indx_{}.h5'.format(k), 'w')
hf.create_dataset('data', data=drop_indx)
hf.close()

hf = h5py.File('prec_max_{}.h5'.format(k), 'w')
hf.create_dataset('data', data=prec_max)
hf.close()

hf = h5py.File('train_prec_{}.h5'.format(k), 'w')
hf.create_dataset('data', data=train_prec)
hf.close()

In [12]:
k=2
if __name__ == '__main__':
    pool = Pool(31)
    print(int(k*12153))
    X_train, X_test, y_train, y_test = train_test_split(
        combo_X, 
        combo_Y, 
        stratify=combo_Y, 
        train_size=0.8,
        test_size=0.2,
        random_state=int(k*12153))

    index_list = np.arange(310)

    prec_max = []
    drop_indx = []
    train_prec = []

    for i in tq.tqdm(range(309),leave=True):
        mask = np.ones(310, dtype=bool)
        mask[drop_indx] = False
        prec_temp = np.zeros(310)
        train_prec_temp = np.zeros(310)
        temp_fnc = partial(mask_prec_fn, mask=mask,X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test)
        out = pool.map(temp_fnc,index_list[mask])
        out1 = list(map(list, zip(*out)))
        
        for indx, ind in enumerate(out1[0]):
            prec_temp[ind] = out1[1][indx]
            train_prec_temp[ind] = out1[2][indx]
        
        arg = np.random.choice(np.flatnonzero(prec_temp == prec_temp.max()))
        drop_indx.append(arg)
        prec_arg = prec_temp[arg]
        prec_max.append(prec_arg)
        train_prec_arg = train_prec_temp[arg]
        train_prec.append(train_prec_arg)

        print(arg,prec_arg,train_prec_arg)
        
    pool.close()
    pool.join()
    
    hf = h5py.File('drop_indx_{}.h5'.format(k), 'w')
    hf.create_dataset('data', data=drop_indx)
    hf.close()

    hf = h5py.File('prec_max_{}.h5'.format(k), 'w')
    hf.create_dataset('data', data=prec_max)
    hf.close()

    hf = h5py.File('train_prec_{}.h5'.format(k), 'w')
    hf.create_dataset('data', data=train_prec)
    hf.close()
    
    k+=1

  0%|          | 0/309 [00:00<?, ?it/s]

24306


  0%|          | 1/309 [00:58<4:59:58, 58.44s/it]

248 0.9287142857142857 0.928625


  1%|          | 2/309 [01:56<4:58:07, 58.26s/it]

...

237 0.5605714285714286 0.5526428571428571


100%|█████████▉| 308/309 [3:07:49<00:02,  2.14s/it]

144 0.4345 0.4323392857142857


100%|██████████| 309/309 [3:07:50<00:00, 36.47s/it]

143 0.30228571428571427 0.295


In [ ]:
k=3
for n in range(10):
    if __name__ == '__main__':
        pool = Pool(31)
        print(int(k*12153))
        X_train, X_test, y_train, y_test = train_test_split(
            combo_X, 
            combo_Y, 
            stratify=combo_Y, 
            train_size=0.8,
            test_size=0.2,
            random_state=int(k*12153))

        index_list = np.arange(310)

        prec_max = []
        drop_indx = []
        train_prec = []

        for i in tq.tqdm(range(309),leave=True):
            mask = np.ones(310, dtype=bool)
            mask[drop_indx] = False
            prec_temp = np.zeros(310)
            train_prec_temp = np.zeros(310)
            temp_fnc = partial(mask_prec_fn, mask=mask,X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test)
            out = pool.map(temp_fnc,index_list[mask])
            out1 = list(map(list, zip(*out)))

            for indx, ind in enumerate(out1[0]):
                prec_temp[ind] = out1[1][indx]
                train_prec_temp[ind] = out1[2][indx]

            arg = np.random.choice(np.flatnonzero(prec_temp == prec_temp.max()))
            drop_indx.append(arg)
            prec_arg = prec_temp[arg]
            prec_max.append(prec_arg)
            train_prec_arg = train_prec_temp[arg]
            train_prec.append(train_prec_arg)

            print(arg,prec_arg,train_prec_arg)

        pool.close()
        pool.join()

        hf = h5py.File('drop_indx_{}.h5'.format(k), 'w')
        hf.create_dataset('data', data=drop_indx)
        hf.close()

        hf = h5py.File('prec_max_{}.h5'.format(k), 'w')
        hf.create_dataset('data', data=prec_max)
        hf.close()

        hf = h5py.File('train_prec_{}.h5'.format(k), 'w')
        hf.create_dataset('data', data=train_prec)
        hf.close()

        k+=1

  0%|          | 0/309 [00:00<?, ?it/s]

36459


  0%|          | 1/309 [01:03<5:27:18, 63.76s/it]

281 0.9307857142857143 0.9285178571428572


  1%|          | 2/309 [02:08<5:27:16, 63.96s/it]

...

47 0.9180714285714285 0.9147321428571429


 74%|███████▍  | 228/309 [2:52:40<46:16, 34.28s/it]

82 0.9175714285714286 0.9143928571428571


 74%|███████▍  | 229/309 [2:53:13<45:09, 33.87s/it]

281 0.9177857142857143 0.914


In [ ]:
k=8
for n in range(10):
    if __name__ == '__main__':
        pool = Pool(31)
        print(int(k*12153))
        X_train, X_test, y_train, y_test = train_test_split(
            combo_X, 
            combo_Y, 
            stratify=combo_Y, 
            train_size=0.8,
            test_size=0.2,
            random_state=int(k*12153))

        index_list = np.arange(310)

        prec_max = []
        drop_indx = []
        train_prec = []

        for i in tq.tqdm(range(309),leave=True):
            mask = np.ones(310, dtype=bool)
            mask[drop_indx] = False
            prec_temp = np.zeros(310)
            train_prec_temp = np.zeros(310)
            temp_fnc = partial(mask_prec_fn, mask=mask,X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test)
            out = pool.map(temp_fnc,index_list[mask])
            out1 = list(map(list, zip(*out)))

            for indx, ind in enumerate(out1[0]):
                prec_temp[ind] = out1[1][indx]
                train_prec_temp[ind] = out1[2][indx]

            arg = np.random.choice(np.flatnonzero(prec_temp == prec_temp.max()))
            drop_indx.append(arg)
            prec_arg = prec_temp[arg]
            prec_max.append(prec_arg)
            train_prec_arg = train_prec_temp[arg]
            train_prec.append(train_prec_arg)

            print(arg,prec_arg,train_prec_arg)

        pool.close()
        pool.join()

        hf = h5py.File('drop_indx_{}.h5'.format(k), 'w')
        hf.create_dataset('data', data=drop_indx)
        hf.close()

        hf = h5py.File('prec_max_{}.h5'.format(k), 'w')
        hf.create_dataset('data', data=prec_max)
        hf.close()

        hf = h5py.File('train_prec_{}.h5'.format(k), 'w')
        hf.create_dataset('data', data=train_prec)
        hf.close()

        k+=1

  0%|          | 0/309 [00:00<?, ?it/s]

97224


  0%|          | 1/309 [01:05<5:36:30, 65.55s/it]

242 0.9262142857142858 0.92925


  1%|          | 2/309 [02:11<5:35:16, 65.52s/it]

...

285 0.9328571428571428 0.9283214285714285


 18%|█▊        | 56/309 [55:43<3:39:44, 52.11s/it]

207 0.9327857142857143 0.9281607142857143


 18%|█▊        | 57/309 [56:34<3:37:02, 51.68s/it]

296 0.933 0.9283035714285715
